In [1]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV, cross_val_score

import pandas as pd
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
X_train = pd.read_csv('../data/X_train_flu.csv', index_col= 'respondent_id')
X_test = pd.read_csv('../data/X_test_flu.csv', index_col= 'respondent_id')
y_train = pd.read_csv('../data/y_train_flu.csv', index_col= 'respondent_id')
y_test = pd.read_csv('../data/y_test_flu.csv', index_col= 'respondent_id')

In [3]:
y_train = y_train['seasonal_vaccine']
y_test = y_test['seasonal_vaccine']

In [4]:
X_train.head()

,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,...,race,sex,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children
respondent_id,,,,,,,,,,,,,,,,,,,,,
7273,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,Other or Multiple,Female,"<= $75,000, Above Poverty",Married,Own,Not in Labor Force,qufhixun,"MSA, Not Principle City",1.0,0.0
13773,2.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,...,White,Female,NaN,Not Married,Own,Employed,lrircsnp,Non-MSA,0.0,0.0
24162,1.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,...,White,Female,"> $75,000",Married,Own,Employed,bhuqouqj,"MSA, Not Principle City",1.0,0.0
17341,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,...,Black,Female,"<= $75,000, Above Poverty",Not Married,Own,Employed,fpwskwrf,"MSA, Principle City",1.0,2.0
23521,2.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,...,White,Male,"<= $75,000, Above Poverty",Married,Own,Not in Labor Force,lzgpxyit,"MSA, Not Principle City",1.0,0.0


In [5]:
X_train.isna().sum()

h1n1_concern                     69
h1n1_knowledge                   85
behavioral_antiviral_meds        51
behavioral_avoidance            155
behavioral_face_mask             14
behavioral_wash_hands            30
behavioral_large_gatherings      55
behavioral_outside_home          55
behavioral_touch_face            90
doctor_recc_h1n1               1605
doctor_recc_seasonal           1605
chronic_med_condition           745
child_under_6_months            617
health_worker                   606
opinion_h1n1_vacc_effective     306
opinion_h1n1_risk               290
opinion_h1n1_sick_from_vacc     293
opinion_seas_vacc_effective     350
opinion_seas_risk               394
opinion_seas_sick_from_vacc     414
age_group                         0
education                      1048
race                              0
sex                               0
income_poverty                 3261
marital_status                 1053
rent_or_own                    1518
employment_status           

In [6]:
y_train.isna().sum()

0

In [7]:
X_train_cat = X_train.select_dtypes('object')
X_train_num = X_train.select_dtypes(['float64', 'int64'])

cat_pipe = Pipeline([
    ('impute', SimpleImputer(strategy='most_frequent')),
    ('ohe', OneHotEncoder(sparse=False, handle_unknown='ignore'))
])
num_pipe = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='most_frequent')),
])

transformer = ColumnTransformer([('categorical', cat_pipe, X_train_cat.columns),
                                 ('numerical', num_pipe, X_train_num.columns)])

In [9]:
model_pipe = Pipeline(steps=[
    ('trans', transformer),
    ('rfc', RandomForestClassifier(verbose=1, n_jobs=-2))
])

model_pipe.fit(X_train, y_train)

[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done  36 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-2)]: Done 100 out of 100 | elapsed:    0.6s finished


Pipeline(steps=[('trans',
                 ColumnTransformer(transformers=[('categorical',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  Index(['age_group', 'education', 'race', 'sex', 'income_poverty',
       'marital_status', 'rent_or_own', 'employment_status', 'hhs_geo_region',
       'census_msa'],
      dtype=...
       'behavioral_touch_face', 'doctor_recc_h1n1', 'doctor_recc_seasonal',
       'chronic_med_condition', 'child_under_6_months', 'health_worker',
       'opinion_h1n1_vacc_effective', 'opinion_h1n1_risk',
   

In [10]:
rfc_cv_score = cross_val_score(model_pipe, X_train, y_train, n_jobs=-1, verbose=3, scoring = 'f1')

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    5.6s remaining:    8.4s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    6.1s finished


In [11]:
rfc_cv_score.mean()

0.7515884531086352

# Random Forest Classifier

In [13]:
model_pipe_2 = Pipeline(steps=[
    ('trans', transformer),
    ('rfc', RandomForestClassifier(verbose=1, n_jobs=-2))
])

In [14]:
params = {
    'rfc__max_depth': list(range(10,100,10)),
    'rfc__criterion': ['gini', 'entropy'],
    'rfc__n_estimators': list(range(100,250,25)),
    'rfc__min_samples_leaf': list(range(2,10,2)),
    'rfc__min_samples_split': list(range(2,10,2)),
}

In [ ]:
gs_rfc = GridSearchCV(model_pipe_2, params, n_jobs=-1, verbose=3, cv = 3, scoring = 'f1')
gs_rfc.fit(X_train, y_train)

In [ ]:
gs_rfc.best_params_

In [ ]:
rfc_cv_1 = cross_val_score(gs_rfc.best_estimator, X_train, y_train, n_jobs=-1, verbose=3, scoring = 'f1')

In [ ]:
rfc_cv_1.mean()

## Gradiant Boost Classifier

In [ ]:
model_pipe_3 = Pipeline(steps=[
    ('trans', transformer),
    ('gbc', GradientBoostingClassifier(verbose=3))
])

In [ ]:
model_pipe_3.get_params()

In [ ]:
params = {
    'gbc__learning_rate': [0.001, 0.01, 0.1, .5, .9],
    'gbc__n_estimators': list(range(100,250,50)),
    'gbc__min_samples_leaf': list(range(2,10,2)),
    'gbc__min_samples_split': list(range(2,10,2)),
    'gbc__max_features': list(range(0,200,50))
}

In [ ]:
gs_gbc = GridSearchCV(model_pipe_3, params, n_jobs=-1, verbose=1, cv = 3, scoring = 'f1')
gs_gbc.fit(X_train, y_train)

In [ ]:
gs_gbc.best_params_

In [ ]:
gbc_cv_1 = cross_val_score(gs_gbc.best_estimator, X_train, y_train, n_jobs=-1, verbose=3, scoring = 'f1')

In [ ]:
gbc_cv_1.mean()